# I) Starting point pour de l'annotation par contours

In [ ]:
!pip install jupyter-dash==0.4.2 dash==2.11.1

In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import requests
from PIL import Image
from io import BytesIO

# Load image
img_url = "https://images.pexels.com/photos/1103970/pexels-photo-1103970.jpeg?auto=compress&cs=tinysrgb&w=640"
response = requests.get(img_url)
image = Image.open(BytesIO(response.content))
w, h = image.size

# Create figure
fig = go.Figure()
fig.add_layout_image(dict(
    source=image, xref="x", yref="y",
    x=0, y=0, sizex=w, sizey=h,
    sizing="stretch", layer="below"
))
fig.update_layout(
    width=w, height=h,
    dragmode="drawclosedpath",
    newshape=dict(line_color="red"),
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis=dict(range=[0, w], visible=False),
    yaxis=dict(range=[h, 0], visible=False)
)

# Global storage
last_polygon = []

def parse_svg_path(path_str):
    import re
    # Extract all x,y pairs using regex
    matches = re.findall(r'([ML])\s*([\d\.\-]+),([\d\.\-]+)', path_str)
    points = [(float(x), float(y)) for _, x, y in matches]
    return points

# Dash app
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Graph(
        id='graph', figure=fig,
        config={'modeBarButtonsToAdd': ['drawclosedpath', 'eraseshape']}
    ),
    html.Div(id='output')
])

@app.callback(
    Output('output', 'children'),
    Input('graph', 'relayoutData'),
    State('graph', 'figure')
)
def display_shape_data(relayout_data, fig_state):
    global last_polygon
    if relayout_data and 'shapes' in relayout_data:
        path = relayout_data['shapes'][-1].get('path', '')
        last_polygon = parse_svg_path(path)
        return html.Pre(f"Polygon points:\n{last_polygon}")
    return "Draw a polygon on the image."

app.run_server(mode='inline')


In [ ]:
print(last_polygon)

In [ ]:
import matplotlib.pyplot as plt

# Reload image
img = Image.open(BytesIO(requests.get(img_url).content))

# Plot image and polygon
plt.imshow(img)
if last_polygon:
    x, y = zip(*last_polygon)
    plt.plot(x + (x[0],), y + (y[0],), color='red', linewidth=2)  # Close the loop
plt.title("Extracted Polygon")
plt.axis('off')
plt.show()